<a href="https://colab.research.google.com/github/Rgc1992/Quiniela/blob/master/Transform_and_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importar librerias

In [ ]:
import pandas as pd
import numpy as np
import glob
import time                
import calendar
import datetime

# Cargar tablas

Cargamos solo los años de 1990-2020

En el dataframe Tabla, tenemos la informacion del equipo desde un punto de vista de la tabla general, mientras que resultados tiene la informacion de cada partido.

In [ ]:
# Importamos todos los datos, en el dataframe Tabla, tenemos la informacion de la tabla de posiciones, mientras que en la tabla Resultado, tenemos la tabla de resultados.
import glob
li = []

for name in sorted(glob.glob('/content/drive/MyDrive/Proyecto_Quini/ETL/DDBB_Tabla/*')):
  data_=pd.read_excel(name)
  data_=data_.iloc[:,[1,2,3,4,5,6,7,8,9,10,11]]
  li.append(data_)
Tabla = pd.concat(li, axis=0, ignore_index=True)

li = []
for name in sorted(glob.glob('/content/drive/MyDrive/Proyecto_Quini/ETL/DDBB_Resultado/*')):
  data_=pd.read_excel(name)
  data_=data_.iloc[:,[1,2,3,4,5,6,7]]
  li.append(data_)
Resultado = pd.concat(li, axis=0, ignore_index=True)

# Cambio de formatos

Modificamos el formato para poder hacer bien el cruce de la informacion entre las distintas tablas.

In [ ]:
# Formatos
Tabla.Jornada=Tabla.Jornada+1
Tabla['Posicion_Str']=Tabla.Posicion.astype(str)
Tabla['Año_Str']=Tabla.Año.astype(str)
Tabla['Jornada_Str']=Tabla.Jornada.astype(str)

Resultado['Año_Str']=Resultado.Año.astype(str)
Resultado['Jornada_Str']=Resultado.Jornada.astype(str)


In [ ]:
Tabla

,Posicion,Equipo,Puntos,Año,Jornada,Ganados,Empatados,Perdidos,Goles_a_favor,Goles_en_contra,Liga,Posicion_Str,Año_Str,Jornada_Str
0,1,Real Burgos CF,2,1990,2,1,0,0,3,0,segunda,1,1990,2
1,2,Sabadell,2,1990,2,1,0,0,1,0,segunda,2,1990,2
2,3,Bilbao Ath.,2,1990,2,1,0,0,2,1,segunda,3,1990,2
3,4,Deportivo,2,1990,2,1,0,0,1,0,segunda,4,1990,2
4,5,Xerez,2,1990,2,1,0,0,4,1,segunda,5,1990,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80786,16,Luton Town,41,1989,39,10,11,17,42,52,premier,16,1989,39
80787,17,Aston Villa,40,1989,39,9,13,16,45,56,premier,17,1989,39
80788,18,Middlesbrough,39,1989,39,9,12,17,44,61,premier,18,1989,39
80789,19,West Ham,38,1989,39,10,8,20,37,62,premier,19,1989,39


Replaces en los nomnbres de los equipos

In [ ]:
Tabla['Equipo']=Tabla['Equipo'].replace('Almería *','Almería').replace('Gimnàstic','Gimnàstic Tarragona').replace('Racing *','Racing').replace("Real Unión Irún","Real Unión de Irún").replace("Ciudad Murcia","Ciudad de Murcia").replace("Sestao Sport Cl.","Sestao Sport Club")

# Generador de Keys

Generamos la Keys necesarias para hacer las uniones.

In [ ]:
# Generamos dos keys. para posteriormente hacer las uniones entre las tablas
Tabla['Key']=Tabla.apply(lambda x:'%s-%s-%s-%s' % (x['Equipo'],x['Jornada_Str'],x['Año_Str'],x['Liga']),axis=1)
Tabla['Key_Año_Liga']=Tabla.apply(lambda x:'%s-%s' % (x['Año_Str'],x['Liga']),axis=1)

Resultado['Key_Local']=Resultado.apply(lambda x:'%s-%s-%s-%s' % (x['Equipo_Local'],x['Jornada_Str'],x['Año_Str'],x['Liga']),axis=1)
Resultado['Key_Visitante']=Resultado.apply(lambda x:'%s-%s-%s-%s' % (x['Equipo_Visitante'],x['Jornada_Str'],x['Año_Str'],x['Liga']),axis=1)
Resultado['Key_Año_Liga']=Resultado.apply(lambda x:'%s-%s' % (x['Año_Str'],x['Liga']),axis=1)



Diccionarios y funciones de transformacion de los datos

In [ ]:
# Diccionarios
Dic_Conteo_Partidos=Resultado.loc[:,['Key_Año_Liga','Jornada']].groupby(by=['Key_Año_Liga']).max().to_dict() # Conteo del numero maximos de jornadas por key_año_liga
Dic_Num_Equipo=(((Resultado.loc[:,['Key_Año_Liga','Jornada']].groupby(by=['Key_Año_Liga']).max())/2)+1).to_dict() # Conteo del numero totales de equipos por key_año_liga
Diccionario_Posiciones_Pos=Tabla.loc[:,['Key','Posicion']].set_index('Key').to_dict() # En funcion de la key, te da la posicion de ese equipo en la tabla
Diccionario_Goles_Favor=Tabla.loc[:,['Key','Goles_a_favor']].set_index('Key').to_dict() # En funcion de la key, te da el numero de goles a favor de ese equipo en esa jornada
Diccionario_Goles_Contra=Tabla.loc[:,['Key','Goles_en_contra']].set_index('Key').to_dict() # En funcion de la key, te da el numero de goles en contra de ese equipo en esa jornada

# Funciones de mapeos

def Posicion(x): # Te da la posicion en funcion de la key
    try:
      x=Diccionario_Posiciones_Pos['Posicion'][x]
    except:
      x='0'
    return x

def gol_favor(x): # Te da los goles a favor en funcion de la key
    try:
      x=Diccionario_Goles_Favor['Goles_a_favor'][x]
    except:
      x=-1
    return x

def gol_en_contra(x): # Te da los goles en contra en funcion de la key
    try:
      x=Diccionario_Goles_Contra['Goles_en_contra'][x]
    except:
      x=-1
    return x

def num_participantes(x): # Te da el numero de participantes en funcion de la key año-liga
    try:
      x=Dic_Num_Equipo['Jornada'][x]
    except:
      x='-1'
    return x

def numero_jornada(x): # Te da el numero de jornadas de ese año, en funcion de la key año-liga
    try:
      x=Dic_Conteo_Partidos['Jornada'][x]
    except:
      x='No'
    return x


# Formato de fecha

def for_fecha(x): # En funcion de la fecha nos da el dia de año.
    try:
      x_spliteado=x.split('-')
      for i in range (0,len(x_spliteado)):
        x_spliteado[i]=x_spliteado[i].replace('Ene','Jan').replace('Abr','Apr').replace('Dic','Dec')
      Fecha=x_spliteado[0]+'-'+x_spliteado[1]+'-'+x_spliteado[2]
      x=datetime.strptime(Fecha,'%d-%b-%Y').strftime('%j')
    except:
      x='No'
    return x

# Resultado 

def Resultado_func(x): # Nos da el resultado 1x2, en funcion del resutado
    try:
      x=x.split('-')
      if (int(x[0])>int(x[1])):
        a='1'
      elif (x[0]==x[1]):
        a='X'
      elif (int(x[0])<int(x[1])):
        a='2'
      else:
        a='TodoMal'
    except:
       a='Mal'
    return a


def Resultado_numerico_local(x):
    try:
      if (x=='1'):
        a=1
      elif (x=='X'):
        a=0
      elif (x=='2'):
        a=-1
      else:
        a='0'
    except:
       a='Mal'
    return a

def Resultado_numerico_visitante(x):
    try:
      if (x=='1'):
        a=-1
      elif (x=='X'):
        a=0
      elif (x=='2'):
        a=+1
      else:
        a='0'
    except:
       a='Mal'
    return a



In [ ]:
Resultado

,Equipo_Local,Equipo_Visitante,Resultado,Año,Jornada,Fecha,Liga,Año_Str,Jornada_Str,Key_Local,Key_Visitante,Key_Año_Liga
0,Racing,Bilbao Ath.,1-2,1990,1,03 Sep 89\nFinalizado,segunda,1990,1,Racing-1-1990-segunda,Bilbao Ath.-1-1990-segunda,1990-segunda
1,Atlético B,Recreativo,2-2,1990,1,03 Sep 89\nFinalizado,segunda,1990,1,Atlético B-1-1990-segunda,Recreativo-1-1990-segunda,1990-segunda
2,Las Palmas,Real Betis,1-0,1990,1,03 Sep 89\nFinalizado,segunda,1990,1,Las Palmas-1-1990-segunda,Real Betis-1-1990-segunda,1990-segunda
3,Espanyol,UD Salamanca,0-1,1990,1,03 Sep 89\nFinalizado,segunda,1990,1,Espanyol-1-1990-segunda,UD Salamanca-1-1990-segunda,1990-segunda
4,UE Figueres,Elche,4-2,1990,1,03 Sep 89\nFinalizado,segunda,1990,1,UE Figueres-1-1990-segunda,Elche-1-1990-segunda,1990-segunda
...,...,...,...,...,...,...,...,...,...,...,...,...
57458,Everton,West Ham,3-1,1989,38,13 May 89\nFinalizado,premier,1989,38,Everton-38-1989-premier,West Ham-38-1989-premier,1989-premier
57459,Luton Town,Norwich City,1-0,1989,38,13 May 89\nFinalizado,premier,1989,38,Luton Town-38-1989-premier,Norwich City-38-1989-premier,1989-premier
57460,Queens Park Rangers,Tottenham Hotspur,1-0,1989,38,13 May 89\nFinalizado,premier,1989,38,Queens Park Rangers-38-1989-premier,Tottenham Hotspur-38-1989-premier,1989-premier
57461,Sheffield Wednesday,Middlesbrough,1-0,1989,38,13 May 89\nFinalizado,premier,1989,38,Sheffield Wednesday-38-1989-premier,Middlesbrough-38-1989-premier,1989-premier


# Data Set Final

In [ ]:
# Transformaciones en funcion a los mappings y las funciones de la tabla de resultados finales

Resultado['Posicion_Local']=pd.DataFrame(Resultado['Key_Local']).applymap(Posicion) # Posicion del equipo local al empezar la jornada
Resultado['Posicion_Visitante']=pd.DataFrame(Resultado['Key_Visitante']).applymap(Posicion) # Posicion del equipo visitante al empezar la jornada
Resultado['Goles_favor_Local']=pd.DataFrame(Resultado['Key_Local']).applymap(gol_favor) # Goles de equipo local al empezar la jornada
Resultado['Goles_favor_Visitante']=pd.DataFrame(Resultado['Key_Visitante']).applymap(gol_favor) # Goles del equipo visitante al empezar la jornada
Resultado['Goles_contra_Local']=pd.DataFrame(Resultado['Key_Local']).applymap(gol_en_contra) # Goles en contra del equipo local al empezar la jornada
Resultado['Goles_contra_Visitante']=pd.DataFrame(Resultado['Key_Visitante']).applymap(gol_en_contra) # Goles en contra, del equipo viositante al comentar la jornada
Resultado['Numero_Participantes']=pd.DataFrame(Resultado['Key_Año_Liga']).applymap(num_participantes) # Numero de participantes por liga y año
Resultado['Fecha_Formato']=Resultado['Fecha'].astype('str').apply(lambda x: str(x[0:2])+'-'+str(x[3:6])+'-') # Primer formato fecha
Resultado['Fecha_Partido']=pd.DataFrame((Resultado['Fecha_Formato']+Resultado['Año_Str'])).applymap(for_fecha) # Formato fecha bueno
Resultado['1x2']=pd.DataFrame(Resultado['Resultado']).applymap(Resultado_func) # Resultado
Resultado['Total_Jornadas']=pd.DataFrame(Resultado['Key_Año_Liga']).applymap(numero_jornada) # Total de jornadas

# Utilizar la buena nomenclatura

Resultado_Final=Resultado
#Resultado_Final=Resultado[Resultado['Posicion_Local']!='Mal']
#Resultado_Final=Resultado_Final[Resultado_Final['Posicion_Visitante']!='Mal']
Resultado_Final['Posicion_Local']=Resultado_Final['Posicion_Local'].astype('int')
Resultado_Final['Posicion_Visitante']=Resultado_Final['Posicion_Visitante'].astype('int')
Resultado_Final['Posicion_Local_Normalizada']=Resultado_Final['Posicion_Local']/Resultado_Final['Numero_Participantes']
Resultado_Final['Posicion_Visitante_Normalizada']=Resultado_Final['Posicion_Visitante']/Resultado_Final['Numero_Participantes']
Resultado_Final['Goles_Local_a_Favor_Normalizados']=Resultado_Final['Goles_favor_Local']/Resultado_Final['Jornada']
Resultado_Final['Goles_Local_en_contra_Normalizados']=Resultado_Final['Goles_contra_Local']/Resultado_Final['Jornada']
Resultado_Final['Goles_Visitante_a_Favor_Normalizados']=Resultado_Final['Goles_favor_Visitante']/Resultado_Final['Jornada']
Resultado_Final['Goles_Visitantes_en_contra_Normalizados']=Resultado_Final['Goles_contra_Visitante']/Resultado_Final['Jornada']
Resultado_Final['Numero_Jornada_Normalizado']=Resultado_Final['Jornada']/Resultado_Final['Total_Jornadas']


Resultado_Final['Valor_Numerico_Resultado_Local']=pd.DataFrame(Resultado_Final['1x2']).applymap(Resultado_numerico_local)
Resultado_Final['Valor_Numerico_Resultado_Visitante']=pd.DataFrame(Resultado_Final['1x2']).applymap(Resultado_numerico_local)

# Racha
Resultado_Final=Resultado_Final.sort_values(by=['Año','Liga','Equipo_Local','Jornada'])

Resultado_Final['Resultado_Anterior_1_Local']=Resultado_Final['Valor_Numerico_Resultado_Local'].shift(1)
Resultado_Final['Resultado_Anterior_2_Local']=Resultado_Final['Valor_Numerico_Resultado_Local'].shift(2)
Resultado_Final['Resultado_Anterior_3_Local']=Resultado_Final['Valor_Numerico_Resultado_Local'].shift(3)

Resultado_Final=Resultado_Final.sort_values(by=['Año','Liga','Equipo_Visitante','Jornada'])

Resultado_Final['Resultado_Anterior_1_Visitante']=Resultado_Final['Valor_Numerico_Resultado_Visitante'].shift(1)
Resultado_Final['Resultado_Anterior_2_Visitante']=Resultado_Final['Valor_Numerico_Resultado_Visitante'].shift(2)
Resultado_Final['Resultado_Anterior_3_Visitante']=Resultado_Final['Valor_Numerico_Resultado_Visitante'].shift(3)

Resultado_Final=Resultado_Final.dropna()
Resultado_Final=Resultado_Final[Resultado_Final.Jornada>6]
Resultado_Final['Racha_Local']=Resultado_Final['Resultado_Anterior_1_Local'].astype('float')+0.5*(Resultado_Final['Resultado_Anterior_2_Local'].astype('float'))+0.2*(Resultado_Final['Resultado_Anterior_3_Local'].astype('float'))
Resultado_Final['Racha_Visitante']=Resultado_Final['Resultado_Anterior_1_Visitante'].astype('float')+0.5*Resultado_Final['Resultado_Anterior_2_Visitante'].astype('float')+0.2*Resultado_Final['Resultado_Anterior_3_Visitante'].astype('float')


# Eliminacion de aquellos registros erroneos, que hace mal el mapeo
Resultado_Final=Resultado_Final[Resultado_Final['Posicion_Local_Normalizada']!=0]
Resultado_Final=Resultado_Final[Resultado_Final['1x2']!='Mal']
 

# Seleccion de las variables

In [ ]:
Datos=Resultado_Final.loc[:,['1x2','Posicion_Local_Normalizada','Posicion_Visitante_Normalizada','Goles_Local_a_Favor_Normalizados','Goles_Local_en_contra_Normalizados','Goles_Visitante_a_Favor_Normalizados','Goles_Visitantes_en_contra_Normalizados','Numero_Jornada_Normalizado','Racha_Local','Racha_Visitante']]
Datos.to_excel("/content/drive/MyDrive/Proyecto_Quini/Modelo/DDBB.xlsx")

In [ ]:
Datos


,1x2,Posicion_Local_Normalizada,Posicion_Visitante_Normalizada,Goles_Local_a_Favor_Normalizados,Goles_Local_en_contra_Normalizados,Goles_Visitante_a_Favor_Normalizados,Goles_Visitantes_en_contra_Normalizados,Numero_Jornada_Normalizado,Racha_Local,Racha_Visitante
23772,1,0.341463,0.243902,1.761905,1.000000,1.952381,1.380952,0.538462,1.5,-0.2
40729,1,0.341463,0.243902,1.761905,1.000000,1.952381,1.380952,0.538462,1.7,1.0
23783,2,0.341463,0.292683,1.136364,1.000000,1.909091,1.409091,0.564103,1.7,1.5
40740,2,0.341463,0.292683,1.136364,1.000000,1.909091,1.409091,0.564103,0.2,0.7
23803,1,0.195122,0.243902,1.666667,0.833333,1.958333,1.291667,0.615385,-1.5,0.2
...,...,...,...,...,...,...,...,...,...,...
20035,3,0.279070,0.558140,1.060606,0.787879,1.393939,1.121212,0.804878,-0.5,0.5
20055,1,0.883721,0.418605,1.114286,1.514286,1.400000,1.057143,0.853659,-1.3,-0.7
20083,1,0.046512,0.511628,1.621622,1.054054,1.432432,1.189189,0.902439,1.2,0.7
20099,2,0.604651,0.511628,0.743590,0.897436,1.384615,1.205128,0.951220,0.7,1.3


In [ ]:
Resultado_Final

,Equipo_Local,Equipo_Visitante,Resultado,Año,Jornada,Fecha,Liga,Año_Str,Jornada_Str,Key_Local,Key_Visitante,Key_Año_Liga,Posicion_Local,Posicion_Visitante,Goles_favor_Local,Goles_favor_Visitante,Goles_contra_Local,Goles_contra_Visitante,Numero_Participantes,Fecha_Formato,Fecha_Partido,1x2,Total_Jornadas,Posicion_Local_Normalizada,Posicion_Visitante_Normalizada,Goles_Local_a_Favor_Normalizados,Goles_Local_en_contra_Normalizados,Goles_Visitante_a_Favor_Normalizados,Goles_Visitantes_en_contra_Normalizados,Numero_Jornada_Normalizado,Valor_Numerico_Resultado_Local,Valor_Numerico_Resultado_Visitante,Resultado_Anterior_1_Local,Resultado_Anterior_2_Local,Resultado_Anterior_3_Local,Resultado_Anterior_1_Visitante,Resultado_Anterior_2_Visitante,Resultado_Anterior_3_Visitante,Racha_Local,Racha_Visitante
23772,Portsmouth,Arsenal,2-1,1950,21,10 Dic 49\nFinalizado,premier,1950,21,Portsmouth-21-1950-premier,Arsenal-21-1950-premier,1950-premier,7,5,37,41,21,29,20.5,10-Dic-,No,1,39,0.341463,0.243902,1.761905,1.000000,1.952381,1.380952,0.538462,1,1,1,1,0,0,0,-1,1.5,-0.2
40729,Portsmouth,Arsenal,2-1,1950,21,10 Dic 49\nFinalizado,premier,1950,21,Portsmouth-21-1950-premier,Arsenal-21-1950-premier,1950-premier,7,5,37,41,21,29,20.5,10-Dic-,No,1,39,0.341463,0.243902,1.761905,1.000000,1.952381,1.380952,0.538462,1,1,1,1,1,1,0,0,1.7,1.0
23783,Burnley,Arsenal,0-0,1950,22,17 Dic 49\nFinalizado,premier,1950,22,Burnley-22-1950-premier,Arsenal-22-1950-premier,1950-premier,7,6,25,42,22,31,20.5,17-Dic-,No,2,39,0.341463,0.292683,1.136364,1.000000,1.909091,1.409091,0.564103,0,0,1,1,1,1,1,0,1.7,1.5
40740,Burnley,Arsenal,0-0,1950,22,17 Dic 49\nFinalizado,premier,1950,22,Burnley-22-1950-premier,Arsenal-22-1950-premier,1950-premier,7,6,25,42,22,31,20.5,17-Dic-,No,2,39,0.341463,0.292683,1.136364,1.000000,1.909091,1.409091,0.564103,0,0,0,0,1,0,1,1,0.2,0.7
23803,Man. Utd,Arsenal,2-0,1950,24,26 Dic 49\nFinalizado,premier,1950,24,Man. Utd-24-1950-premier,Arsenal-24-1950-premier,1950-premier,4,5,40,47,20,31,20.5,26-Dic-,No,1,39,0.195122,0.243902,1.666667,0.833333,1.958333,1.291667,0.615385,1,1,-1,-1,0,0,0,1,-1.5,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20035,Osasuna,Tenerife,0-1,2018,33,01 Abr 18\n16:00 Finalizado,segunda,2018,33,Osasuna-33-2018-segunda,Tenerife-33-2018-segunda,2018-segunda,6,12,35,46,26,37,21.5,01-Abr-,No,3,41,0.279070,0.558140,1.060606,0.787879,1.393939,1.121212,0.804878,-1,-1,-1,1,0,1,-1,0,-0.5,0.5
20055,Cultural Leonesa,Tenerife,3-2,2018,35,14 Abr 18\n20:00 Finalizado,segunda,2018,35,Cultural Leonesa-35-2018-segunda,Tenerife-35-2018-segunda,2018-segunda,19,9,39,49,53,37,21.5,14-Abr-,No,1,41,0.883721,0.418605,1.114286,1.514286,1.400000,1.057143,0.853659,1,1,-1,-1,1,-1,1,-1,-1.3,-0.7
20083,Rayo Vallecano,Tenerife,3-1,2018,37,29 Abr 18\n18:00 Finalizado,segunda,2018,37,Rayo Vallecano-37-2018-segunda,Tenerife-37-2018-segunda,2018-segunda,1,11,60,53,39,44,21.5,29-Abr-,No,1,41,0.046512,0.511628,1.621622,1.054054,1.432432,1.189189,0.902439,1,1,1,0,1,1,-1,1,1.2,0.7
20099,Reus Deportiu,Tenerife,1-1,2018,39,12 May 18\n20:30 Finalizado,segunda,2018,39,Reus Deportiu-39-2018-segunda,Tenerife-39-2018-segunda,2018-segunda,13,11,29,54,35,47,21.5,12-May-,No,2,41,0.604651,0.511628,0.743590,0.897436,1.384615,1.205128,0.951220,0,0,0,1,1,1,1,-1,0.7,1.3
